In [35]:
import numpy as np

## Exact partition function of Ising model on $L\times M$ lattice. 

The `log_Z` function below returns the log of the partition function of Ising model on a rectangular L x M lattice per one lattice site. [Ferdinand, A. E., & Fisher, M. E. (1969). Bounded and Inhomogeneous Ising Models. I. Specific-Heat Anomaly of a Finite Lattice. Physical Review, 185(2), 832. ](https://doi.org/10.1103/PhysRev.185.832)

$$\frac{1}{L\times M}\log Z(\beta,L,M)$$

$$Z=\sum_s e^{-\beta E(s)}$$

$$E(s)=-\sum_{\langle i,j\rangle} s_i\cdot s_j,\quad s_i=\pm 1$$

In [36]:
def log_prod(factors):
    
    sign = np.prod(np.sign(factors)) 
    return np.sum(np.log(np.abs(factors))), sign
                                       

def log_Z(beta,L, M):
    

    rs = np.arange(2*M)

    cs = np.cosh(2*beta)**2/np.sinh(2*beta) - np.cos(rs*np.pi/M)
    csp= cs[1:]
    gs = np.empty_like(cs)
    gs[1:] = np.log(csp + np.sqrt(csp*csp-1))
    gs[0] = 2 * beta +np.log(np.tanh(beta)) 

   
    
    z1 = np.prod(np.cosh(0.5*L*gs[1::2]))
    z2 = np.prod(np.sinh(0.5*L*gs[1::2]))
    
    z3 = np.prod(np.cosh(0.5*L*gs[::2]) )
    z4 = np.prod(np.sinh(0.5*L*gs[::2]) )
    
    log_z1,s1 = log_prod(np.cosh(0.5*L*gs[1::2]))
    log_z2,s2 = log_prod(np.sinh(0.5*L*gs[1::2]))
    
    log_z3,s3 = log_prod(np.cosh(0.5*L*gs[::2]) )
    log_z4,s4 = log_prod(np.sinh(0.5*L*gs[::2]) )
    
    log_zi = np.array([log_z1,log_z2,log_z3,log_z4])
    si = np.array([s1,s2,s3,s4])
   
    max_log = np.max(log_zi)
    
    log_z = max_log + np.log(np.sum(np.exp(log_zi-max_log)*si))

    return ((M+L*M/2)*np.log(2) + (L*M/2)*np.log(np.sinh(2*beta))-np.log(2) + log_z)/(L*M)



## Entanglement 

We assume two copies of a  rectangular $L\times k L$ lattice. We glue the copies in such a way that that $l$ spins (part A) on the border have periodic boundary conditions on each copy separately and remaining $L-l$ spins (part B) are glued together on the border.
To calculate the partition function of such a system that we will call $Z_2(\beta,L,k,l)$ we will use the transition matrix approach. 

## Transition matrix

We will start with the partition function of an Ising model on a  $L\times M$  lattice which can can be written as

$\newcommand{\tr}{\operatorname{Tr}}$
$$Z(\beta,L,M)=\tr T^M(\beta, L)$$

where $T(\beta,L)$ is a _transition matrix_ of size $2^L\times2^L$. The element of this matrix between two chains of spins $s$ and $s'$ of length $L$ is given by

\begin{equation}
\begin{split}
    T_{s';s}=\exp\left(
    \beta\frac{1}{2}\sum_{i=0}^{L-1} \left(s_i\cdot s_{i+1}+s'_i\cdot s'_{i+1}\right)+\beta\sum_{i=0}^{L-1} s_i\cdot s'_i
    \right)
\end{split}
\end{equation}

The factor one half in the first term of the exponential prevents the double counting of the _self energy_ of each chain.  The function below implements this formula 

In [37]:
def T(beta,s_i, s_f):
    e = 0.5*np.sum(np.roll(s_i,1)*s_i)
    e += 0.5*np.sum(np.roll(s_f,1)*s_f)
    e +=np.dot(s_f,s_i)
    return np.exp(beta*e)

We will use the critical value of $\beta$

$$\beta_c=\frac{1}{2}\log\left(1+\sqrt{2}\right)$$

In [38]:
bc = 0.5*np.log(1+np.sqrt(2))
bc

0.44068679350977147

In [39]:
L = 8

To generate the transition matrix we need to enumerate all the possible  states of $L$ spins. Function below takes a list of values and returns an array of all combinations of $n$ such values. If we call the set of values $V$ then this function is generating the Cartesian product 

$$V^{n}=\underbrace{V\times\cdots\times V}_n$$

In [40]:
def cartesian(values, n):
    arrays = [values] * n
    return np.array(np.meshgrid(*arrays, indexing='ij')).T.reshape(-1, n)

In [41]:
states = cartesian([-1,1],L)

In [42]:
states.shape

(256, 8)

In [43]:
states[:4]

array([[-1, -1, -1, -1, -1, -1, -1, -1],
       [ 1, -1, -1, -1, -1, -1, -1, -1],
       [-1,  1, -1, -1, -1, -1, -1, -1],
       [ 1,  1, -1, -1, -1, -1, -1, -1]])

We will also need function that transform the states into integer indices

In [44]:
two_pows = 2**np.arange(0,L)
def state_to_int(s):
    return np.sum( ((s+1)//2)*two_pows )

Given the states we can generate the transition matrix

In [45]:
%%time
T_mat = np.empty((len(states), len(states)) )
for s_i in states:
    for s_j in states:
        T_mat[state_to_int(s_i),state_to_int(s_j)]=T(bc, s_i,s_j)

CPU times: user 2.08 s, sys: 0 ns, total: 2.08 s
Wall time: 2.08 s


This matrix can contain some large elements so to avoid the overflow we will factor out the largest element $C=\exp(2 \beta L)$

In [46]:
C = np.exp(2*bc*L)
print(C)

1153.999133448222


In [47]:
T_mat/=C

The transition matrix $T$ is symmetric and we can represent it as 

$$T = P\cdot D \cdot P^T$$

where $D$ is a diagonal matrix and $P$ is an orthogonal matrix. The diagonal of matrix $D$ consists of the eigenvalues of matrix $T$ and each column of matrix $P$ is a corresponding eigenvector. 

In [48]:
values, P =  np.linalg.eigh(T_mat)

With such representation the power of the matrix is given by

$$T^{k L} = P\cdot D^{ k L} \cdot P^T$$

Unfortunately using this matrix would give us incorrect gluing at the replicas border. What we need is a matrix that takes into account  self energy of only  one row: 

\begin{equation}
\begin{split}
    A_{s';s}=\exp\left(
    \beta\sum_{i=0}^{L-1} s'_i\cdot s'_{i+1} + \beta\sum_{i=0}^{L-1} s_i\cdot s'_i
    \right)
\end{split}
\end{equation}

please note that this time we do not need the one half factor. This matrix is asymmetric  which makes  calculations more difficult. However this matrix can be reconstructed from the symmetric matrix $T$.

$$A_{s';s} = p_{s'}T_{s';s}p^{-1}_{s}$$

where

$$p_{s'}=\exp\left(
    \beta\frac{1}{2}\sum_{i=0}^{L-1} s'_i\cdot s'_{i+1} \right)
$$    

Please note that we are not using the summation convention in this case. No summation over $s'$ and $s$ is implied. 

In [49]:
def p(beta,s):
    e = 0.5*np.sum(np.roll(s,1)*s)
    return np.exp(beta*e)

In [50]:
p_vec = np.empty(len(states))
for s in states:
    p_vec[state_to_int(s)]=p(bc, s)

It is easy to check that 

$$A^{k L}_{s';s} = \sum_{k}p_{s'}P_{s';t}\cdot D^{ k L}_{t;t} \cdot P^T_{t;s}p^{-1}_s$$

Let's fix the values of $k$ and $l$

In [51]:
k = 8
l = 3

In [52]:
A_kL = p_vec.reshape(-1,1)*P@np.diag(values**(k*L))@P.T/p_vec

Given that the partition function of a single copy is the trace of this matrix

In [53]:
Z_kL = np.trace(A_kL)

Remembering that we have factored out $C=\exp(2\beta L)$ and that we have to factor it back in we finally obtain the logarithm of $Z$ per lattice site 

In [54]:
log_Z_kL = (np.log(Z_kL)+k*L*2*bc*L)/(L*L*k)
log_Z_kL

0.93382806691014

which can be compared to the exact value

In [55]:
log_Z(bc,L,k*L)

0.9338280669101398

### Replicas

To calculated the partition function of the two replicas with the special gluing we have to split the indices of the matrix $A$ into to parts  $A$ consisting of $l$ spins and $B$ containing the remaining $L-l$.

$$s=s_A,s_B$$

With such split the partition function of the two replicas can be written as 

\begin{equation}
\begin{split}
    Z^{(2)}(k) &= \sum_{s_A, s_B, s'_A, s'_B} A^{k L}_{s'_A,s_B;s'_A,s'_B} A^{k L}_{s_A,s'_B;s_A,s_B}\\
    &= \sum_{s'_B} A^{k L}_{B:s_B;s'_B} A^{k L}_{B:s'_B;s_B}
\end{split}    
\end{equation}

where

\begin{equation}
    A^{k L}_{B:s'_B;s_B}\equiv \sum_{s_A} A^{k L}_{s_A,s'_B;s_A,s_B}
\end{equation}

Actually the $A_B$ matrix can be easily calculated using the `reshape` function. What this function does it the write the $2^L\times 2^L$ matrix as four dimensional 
$2^l\times 2^{L-l}\times2^l\times 2^{L-l}$ tensor.

$$A^{k L}_{s';s}=A^{k L}_{s'_A,s'_B,s_A,s_B}$$

After doing that we obtain $A_B$ by taking the trace over first and third index

In [56]:
A_B = np.trace(A_kL.reshape(2**l,2**(L-l),2**l,2**(L-l)),axis1=0, axis2=2)

and we obtain a $2^{L-l}\times 2^{L-l}$ matrix

In [57]:
A_B.shape

(32, 32)

Final value is obtained by taking the trace of $A_B^2$

In [58]:
Z_2_kL = np.trace(A_B@A_B)
Z_2_kL

1.3955543120149802e+23

In [59]:
log_Z_2_kL = (np.log(Z_2_kL)+2*k*L*np.log(C))/(2*k*L*L)
print(log_Z_2_kL)

0.9334172870502992


We can calculate the ratio 

$$\frac{Z_2(\beta,L,k,l)}{Z^2(\beta,L,k)}$$

directly

In [60]:
Z_2_kL/(Z_kL*Z_kL)

0.6566273791083699

or using logarithms

In [61]:
2*L*L*k*(log_Z_2_kL-log_Z_kL)

-0.42063857647690384

In [62]:
np.exp(_)

0.656627379108436

## k = infinity

When $k$ goes to infinity the only term that survives is the largest eigenvalue and the corresponding eigenvector. Actually the single eigenvalues will drop out from the calculations so

In [63]:
A_kL= p_vec.reshape(-1,1)*np.outer(P[:,0],P[:,0])/p_vec

In [64]:
A_B = np.trace(A_kL.reshape(2**l,2**(L-l),2**l,2**(L-l)),axis1=0, axis2=2)

In [65]:
A_B.shape

(32, 32)

In [66]:
Z_2kL = np.trace(A_B@A_B)
Z_2kL

0.6574039924885158

In [67]:
Z = np.trace(A_kL)
Z

0.9999999999999998

In [68]:
Z_2kL/(Z*Z)

0.6574039924885161

Which compares rather well with results of the large $k$ calculations above.